# EDA of Claim Dataaset
# 02_EDA_Claim

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 24/09/2025   | Adrienne | Created   | Created to do EDA for Claim Dataset | 
|    | |   | |

# Content

* [Introduction](#introduction)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import json_lines

In [3]:
# readin clean datafiles
path = "../data/clean/"
claim_df = pd.read_pickle(path + 'claim_mini_sample.pkl')
#claim_df =  pd.read_pickle(path + 'claim.pkl')
#claim_response_df =  pd.read_pickle(path + 'claim_response.pkl')

This section explores dropping columns from Claim

In [ ]:
# Remove all columns that have system or url in the name
drop_cols = claim_df.columns[claim_df.columns.str.contains("system|url")]
claim_fil_df = claim_df.drop(drop_cols, axis = 1)

In [52]:
print(len(claim_df.columns))
print(len(claim_fil_df.columns))

7434
3856


In [11]:
claim_fil_df.head()

,billablePeriod_end,billablePeriod_start,contained_0_birthDate,contained_0_extension_0_valueCode,contained_0_gender,contained_0_id,contained_0_identifier_0_type_coding_0_code,contained_0_identifier_0_type_coding_0_display,contained_0_identifier_0_value,contained_0_name_0_family,...,item_153_detail_0_productOrService_coding_0_code,item_153_detail_0_quantity_code,item_153_detail_0_quantity_unit,item_153_detail_0_quantity_value,item_153_detail_0_sequence,patient_medicare_number,patient_number,patient_first_name,patient_last_name,unique_claim_ID
126283,2019-06-21,2019-06-21,1957-02-11,248152002,female,patient,MC,Patient's Medicare Number,1S00E00MQ11,Quigley282,...,NaN,NaN,NaN,NaN,NaN,1S00E00MQ11,#patient,Leatrice18,Quigley282,100234871
125505,2019-09-19,2019-09-19,1944-09-15,248153007,male,patient,MC,Patient's Medicare Number,1S00E00HX51,Kovacek682,...,NaN,NaN,NaN,NaN,NaN,1S00E00HX51,#patient,Francisco4,Kovacek682,100100893
78173,2014-03-01,2014-03-01,1953-07-02,248152002,female,patient,MC,Patient's Medicare Number,1S00E00GJ31,Quigley282,...,NaN,NaN,NaN,NaN,NaN,1S00E00GJ31,#patient,Leigha840,Quigley282,100016589
116465,2021-02-19,2021-02-19,1943-07-02,248153007,male,patient,MC,Patient's Medicare Number,1S00E00AJ29,Gleichner915,...,NaN,NaN,NaN,NaN,NaN,1S00E00AJ29,#patient,Reed154,Gleichner915,100126542
165747,2014-02-04,2014-02-04,1955-08-14,248153007,male,patient,MC,Patient's Medicare Number,1S00E00MV51,Sosa986,...,NaN,NaN,NaN,NaN,NaN,1S00E00MV51,#patient,JulioC\u00e9sa,Sosa986,100244098


In [21]:
diagnosis_cols = sorted([col for col in claim_fil_df.columns if 'diagnosis' in col])
diagnosis_cols

['diagnosis',
 'diagnosis_0_diagnosisCodeableConcept_coding_0_code',
 'diagnosis_0_sequence',
 'diagnosis_0_type_0_coding_0_code',
 'diagnosis_0_type_0_coding_0_display',
 'diagnosis_0_type_1_coding_0_code',
 'diagnosis_0_type_1_coding_0_display',
 'diagnosis_10_diagnosisCodeableConcept_coding_0_code',
 'diagnosis_10_sequence',
 'diagnosis_10_type_0_coding_0_code',
 'diagnosis_10_type_0_coding_0_display',
 'diagnosis_10_type_1_coding_0_code',
 'diagnosis_10_type_1_coding_0_display',
 'diagnosis_11_diagnosisCodeableConcept_coding_0_code',
 'diagnosis_11_sequence',
 'diagnosis_11_type_0_coding_0_code',
 'diagnosis_11_type_0_coding_0_display',
 'diagnosis_11_type_1_coding_0_code',
 'diagnosis_11_type_1_coding_0_display',
 'diagnosis_12_diagnosisCodeableConcept_coding_0_code',
 'diagnosis_12_sequence',
 'diagnosis_12_type_0_coding_0_code',
 'diagnosis_12_type_0_coding_0_display',
 'diagnosis_12_type_1_coding_0_code',
 'diagnosis_12_type_1_coding_0_display',
 'diagnosis_13_diagnosisCodeable

In [44]:
item_cols = sorted([col for col in claim_fil_df.columns if 'item' in col])
item_cols[-900:-450]

['item_37_revenue_extension_0_valueCodeableConcept_coding_0_code',
 'item_37_sequence',
 'item_37_servicedDate',
 'item_38_detail_0_productOrService_coding_0_code',
 'item_38_detail_0_quantity_code',
 'item_38_detail_0_quantity_unit',
 'item_38_detail_0_quantity_value',
 'item_38_detail_0_sequence',
 'item_38_extension_0_valueQuantity_value',
 'item_38_extension_1_valueCodeableConcept_coding_0_code',
 'item_38_productOrService_coding_0_code',
 'item_38_productOrService_extension_0_valueCodeableConcept_coding_0_code',
 'item_38_revenue_coding_0_code',
 'item_38_revenue_extension_0_valueCodeableConcept_coding_0_code',
 'item_38_sequence',
 'item_38_servicedDate',
 'item_39_detail_0_productOrService_coding_0_code',
 'item_39_detail_0_quantity_code',
 'item_39_detail_0_quantity_unit',
 'item_39_detail_0_quantity_value',
 'item_39_detail_0_sequence',
 'item_39_extension_0_valueQuantity_value',
 'item_39_extension_1_valueCodeableConcept_coding_0_code',
 'item_39_productOrService_coding_0_cod

In [45]:
for i in item_cols:
    print(i)

item
item_0_detail_0_productOrService_coding_0_code
item_0_detail_0_quantity_code
item_0_detail_0_quantity_unit
item_0_detail_0_quantity_value
item_0_detail_0_sequence
item_0_extension_0_valueCodeableConcept_coding_0_code
item_0_extension_0_valueQuantity_value
item_0_extension_1_valueCodeableConcept_coding_0_code
item_0_productOrService_coding_0_code
item_0_productOrService_extension_0_valueCodeableConcept_coding_0_code
item_0_revenue_coding_0_code
item_0_revenue_extension_0_valueCodeableConcept_coding_0_code
item_0_sequence
item_0_servicedDate
item_0_servicedPeriod_end
item_0_servicedPeriod_start
item_100_detail_0_productOrService_coding_0_code
item_100_detail_0_quantity_code
item_100_detail_0_quantity_unit
item_100_detail_0_quantity_value
item_100_detail_0_sequence
item_100_extension_0_valueQuantity_value
item_100_extension_1_valueCodeableConcept_coding_0_code
item_100_productOrService_coding_0_code
item_100_productOrService_extension_0_valueCodeableConcept_coding_0_code
item_100_rev

In [49]:
claim_fil_df.item_34_productOrService_coding_0_code.unique()

array(['G0299', nan, 'G0158', 'S9131', 'G0129', 'G0155', 'G0152', 'G0153',
       'S9126', 'G0156', 'Q5001', 'G0157', 'G0154', 'G0151', 'S9129',
       'G0300', 'G9857'], dtype=object)

In [50]:
service_cols = [col for col in claim_fil_df.columns if 'productOrService_coding_0_code' in col]

In [51]:
len(service_cols)

504

In [53]:
ext_cols = sorted([col for col in claim_fil_df.columns if 'extension' in col])
for i in ext_cols:
    print(i)

contained_0_extension_0_valueCode
contained_1_extension_0_valueCoding_code
contained_1_extension_1_valueCoding_code
extension_0_valueCoding_code
facility_extension_0_valueCoding_code
item_0_extension_0_valueCodeableConcept_coding_0_code
item_0_extension_0_valueQuantity_value
item_0_extension_1_valueCodeableConcept_coding_0_code
item_0_productOrService_extension_0_valueCodeableConcept_coding_0_code
item_0_revenue_extension_0_valueCodeableConcept_coding_0_code
item_100_extension_0_valueQuantity_value
item_100_extension_1_valueCodeableConcept_coding_0_code
item_100_productOrService_extension_0_valueCodeableConcept_coding_0_code
item_100_revenue_extension_0_valueCodeableConcept_coding_0_code
item_101_extension_0_valueQuantity_value
item_101_extension_1_valueCodeableConcept_coding_0_code
item_101_productOrService_extension_0_valueCodeableConcept_coding_0_code
item_101_revenue_extension_0_valueCodeableConcept_coding_0_code
item_102_extension_0_valueQuantity_value
item_102_extension_1_valueCo

In [54]:
len(ext_cols)

1406